In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5.csv')
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
0,0,17163092,도전최강달인왕만두,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,134082,5224.0,NaN,NaN,NaN,127.159471,37.556197,5호선 상일동,373.362877,NaN
1,12,28480888,옥돌영양탕,NaN,Q,음식,Q05,닭/오리요리,Q05A01,삼계탕전문,...,143220,4945.0,NaN,1,NaN,127.089673,37.557977,5호선 아차산(어린이대공원후문),354.921957,NaN
2,14,17175285,은진이네,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,121861,4206.0,NaN,NaN,NaN,126.956164,37.551340,5호선 애오개,53.083100,NaN
3,15,17175333,흥부보쌈,NaN,Q,음식,Q01,한식,Q01A08,족발/보쌈전문,...,158721,7997.0,NaN,5,NaN,126.875987,37.529132,5호선 오목교(목동운동장앞),353.540138,NaN
4,28,20665661,가파도참치,NaN,Q,음식,Q03,일식/수산물,Q03A04,참치전문점,...,158050,8005.0,NaN,2,NaN,126.873242,37.524340,5호선 오목교(목동운동장앞),122.836352,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20044,135800,17212093,만랩커피,오금역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,138800,5702.0,NaN,NaN,103,127.125353,37.502381,5호선 오금,124.273859,NaN
20045,135805,17218896,얼음과자,NaN,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,157857,7510.0,NaN,1,123,126.812418,37.577436,5호선 방화,27.430119,NaN
20046,135806,17218871,오이소서울,상일점,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,134090,5276.0,NaN,1,104,127.165574,37.550924,5호선 상일동,983.009054,NaN
20047,135807,17218873,얼음왕국명일14,6호점,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,134780,5297.0,상가,1,106,127.146217,37.546317,5호선 굽은다리(강동구민회관앞),347.273163,NaN


In [3]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
0,도전최강달인왕만두,한식,한식/백반/한정식,서울특별시 강동구 고덕로 333,5호선 상일동,373.362877,NaN,상일동,상일동역 도전최강달인왕만두
1,옥돌영양탕,닭/오리요리,삼계탕전문,서울특별시 광진구 긴고랑로36길 24,5호선 아차산(어린이대공원후문),354.921957,NaN,아차산,아차산역 옥돌영양탕
2,은진이네,한식,한식/백반/한정식,서울특별시 마포구 마포대로 196,5호선 애오개,53.083100,NaN,애오개,애오개역 은진이네
3,흥부보쌈,한식,족발/보쌈전문,서울특별시 양천구 목동동로 309,5호선 오목교(목동운동장앞),353.540138,NaN,오목교,오목교역 흥부보쌈
4,가파도참치,일식/수산물,참치전문점,서울특별시 양천구 오목로54길 2,5호선 오목교(목동운동장앞),122.836352,NaN,오목교,오목교역 가파도참치
...,...,...,...,...,...,...,...,...,...
20044,만랩커피,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 중대로23길 21,5호선 오금,124.273859,NaN,오금,오금역 만랩커피
20045,얼음과자,패스트푸드,아이스크림판매,서울특별시 강서구 금낭화로 135,5호선 방화,27.430119,NaN,방화,방화역 얼음과자
20046,오이소서울,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로80길 123,5호선 상일동,983.009054,NaN,상일동,상일동역 오이소서울
20047,얼음왕국명일14,패스트푸드,아이스크림판매,서울특별시 강동구 상암로 214,5호선 굽은다리(강동구민회관앞),347.273163,NaN,굽은다리,굽은다리역 얼음왕국명일14


In [6]:
items = data["검색어"][2158:]
items

2158          마포역 도화동지니
2159          개롱역 설악칡냉면
2160       광화문역 장수한방삼계탕
2161        군자역 갯마을새벽시장
2162           장한평역 원성집
              ...      
20044          오금역 만랩커피
20045          방화역 얼음과자
20046        상일동역 오이소서울
20047    굽은다리역 얼음왕국명일14
20048         상일동역 얼음과자
Name: 검색어, Length: 17891, dtype: object

In [7]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}


In [8]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path=r'C:\Users\정예림\Desktop\텍스트마이닝\chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img ') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url[0].text}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url[0].text}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img ') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url[0].text}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url[0].text}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        print("식당 존재 x")


진행상황 : 1/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2/17891
식당 존재 x
진행상황 : 3/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 4/17891
식당 존재 x
진행상황 : 5/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 6/17891
식당 존재 x
진행상황 : 7/17891
식당 존재 x
진행상황 : 8/17891
식당 존재 x
진행상황 : 9/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 10/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 11/17891
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 12/17891
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 13/17891
식당 존재 x
진행상황 : 14/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 15/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 16/17891
식당 존재 x
진행상황 : 17/17891
식당 존재 x
진행상황 : 18/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 19/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 20/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 21/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 22/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 23/17891
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 24/17891
식당 존재 x
진행상황 : 

식당 존재 x
진행상황 : 197/17891
식당 존재 x
진행상황 : 198/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 199/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 200/17891
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 201/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 202/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 203/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 204/17891
식당 존재 x
진행상황 : 205/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 206/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 207/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 208/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 209/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 210/17891
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 211/17891
식당 존재 x
진행상황 : 212/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 213/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 214/17891
식당 존재 x
진행상황 : 215/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 216/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 217/17891
식당 존재
3
더 이상 리뷰 존재 X
기본

식당 존재 x
진행상황 : 381/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 382/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 383/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 384/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 385/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 386/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 387/17891
식당 존재 x
진행상황 : 388/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 389/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 390/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 391/17891
식당 존재 x
진행상황 : 392/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 393/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 394/17891
식당 존재 x
진행상황 : 395/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 396/17891
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 397/17891
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 398/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 399/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 400/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌

더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 565/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 566/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 567/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 568/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 569/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 570/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 571/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 572/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 573/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 574/17891
식당 존재 x
진행상황 : 575/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 576/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 577/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 578/17891
식당 존재 x
진행상황 : 579/17891
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 580/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 581/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 582/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 583/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 584/17891
식당 존재
5
더 이상 

식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 743/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 744/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 745/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 746/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 747/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 748/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 749/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 750/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 751/17891
식당 존재 x
진행상황 : 752/17891
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 753/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 754/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 755/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 756/17891
식당 존재 x
진행상황 : 757/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 758/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 759/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 760/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 761/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 

...

더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1452/17891
식당 존재 x
진행상황 : 1453/17891
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1454/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1455/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1456/17891
식당 존재 x
진행상황 : 1457/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1458/17891
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1459/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1460/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1461/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1462/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1463/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1464/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1465/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1466/17891
식당 존재 x
진행상황 : 1467/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1468/17891
식당 존재 x
진행상황 : 1469/17891
식당 존재 x
진행상황 : 1470/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1471/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1472/17891
식당 존재
4
더 이상 리뷰 존재 

0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1627/17891
식당 존재 x
진행상황 : 1628/17891
식당 존재 x
진행상황 : 1629/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1630/17891
식당 존재 x
진행상황 : 1631/17891
식당 존재 x
진행상황 : 1632/17891
식당 존재 x
진행상황 : 1633/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1634/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1635/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1636/17891
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1637/17891
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1638/17891
식당 존재 x
진행상황 : 1639/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1640/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1641/17891
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1642/17891
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1643/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1644/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1645/17891
식당 존재 x
진행상황 : 1646/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1647/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1648/17891
식당 존재
5


더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1803/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1804/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1805/17891
식당 존재 x
진행상황 : 1806/17891
식당 존재 x
진행상황 : 1807/17891
식당 존재 x
진행상황 : 1808/17891
식당 존재 x
진행상황 : 1809/17891
식당 존재 x
진행상황 : 1810/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1811/17891
식당 존재 x
진행상황 : 1812/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1813/17891
식당 존재 x
진행상황 : 1814/17891
식당 존재 x
진행상황 : 1815/17891
식당 존재 x
진행상황 : 1816/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1817/17891
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1818/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1819/17891
식당 존재 x
진행상황 : 1820/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1821/17891
식당 존재 x
진행상황 : 1822/17891
식당 존재 x
진행상황 : 1823/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1824/17891
식당 존재 x
진행상황 : 1825/17891
식당 존재 x
진행상황 : 1826/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1827/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지

식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1989/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1990/17891
식당 존재 x
진행상황 : 1991/17891
식당 존재 x
진행상황 : 1992/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1993/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1994/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1995/17891
식당 존재 x
진행상황 : 1996/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1997/17891
식당 존재 x
진행상황 : 1998/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1999/17891
식당 존재 x
진행상황 : 2000/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2001/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2002/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2003/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2004/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2005/17891
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2006/17891
식당 존재 x
진행상황 : 2007/17891
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2008/17891
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2009/17891
식당 존재 x
진행상황 : 2010/17891
식당

KeyboardInterrupt: 

In [9]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,10jgadh,마포역 도화동지니,5점,2021.03.04.
1,n2cj41,마포역 도화동지니,3점,2021.01.27.
2,e1t7fd,마포역 도화동지니,4점,2020.09.30.
3,bkki64,마포역 도화동지니,5점,2020.07.11.
4,fjtuhl,마포역 도화동지니,4점,2019.11.26.
...,...,...,...,...
8889,None,우장산역 왕세숫대야냉면,None,2015.01.04.
8890,None,우장산역 왕세숫대야냉면,None,2013.08.26.
8891,None,우장산역 왕세숫대야냉면,None,2013.08.16.
8892,None,우장산역 왕세숫대야냉면,None,2013.08.16.


In [30]:
img_elem

,ItemID,img_url
0,마포역 도화동지니,None
1,마포역 도화동지니,
2,마포역 도화동지니,None
3,마포역 도화동지니,None
4,마포역 도화동지니,None
...,...,...
8889,우장산역 왕세숫대야냉면,None
8890,우장산역 왕세숫대야냉면,
8891,우장산역 왕세숫대야냉면,None
8892,우장산역 왕세숫대야냉면,None


In [11]:
review_elem

,ItemID,review
0,마포역 도화동지니,진짜 맛있어요 !!!
1,마포역 도화동지니,스테이크 맛있다 해서 갔는데 퍽퍽하고 질겼어요 ㅜㅜ 사장님과 직원분들은 친절하심
2,마포역 도화동지니,싼마이에 갈만해요
3,마포역 도화동지니,최고예요
4,마포역 도화동지니,좋아요
...,...,...
8889,우장산역 왕세숫대야냉면,"돈까스, 냉면"
8890,우장산역 왕세숫대야냉면,"세상에서 가격대비 만족도최고 왕냉 면과 돈가스,쫄면 3~4천원으로 깊은맛을 느낄만한..."
8891,우장산역 왕세숫대야냉면,화곡동에 여기 만한 돈까슨 없오
8892,우장산역 왕세숫대야냉면,돈까스 한가득!


In [19]:
img_elem.head()

,ItemID,img_url
0,마포역 도화동지니,None
1,마포역 도화동지니,
2,마포역 도화동지니,None
3,마포역 도화동지니,None
4,마포역 도화동지니,None


In [20]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




      UserID        ItemID Rating    Timestamp
8833  bee73c  우장산역 왕세숫대야냉면     3점  2021.04.04.
8834  sa540j  우장산역 왕세숫대야냉면     5점  2021.04.02.
8835  ndhfg3  우장산역 왕세숫대야냉면     1점  2021.04.02.
8836  a3pqr8  우장산역 왕세숫대야냉면     5점  2021.02.22.
8837  7kc4tr  우장산역 왕세숫대야냉면     4점  2021.02.22.
...      ...           ...    ...          ...
8889    None  우장산역 왕세숫대야냉면   None  2015.01.04.
8890    None  우장산역 왕세숫대야냉면   None  2013.08.26.
8891    None  우장산역 왕세숫대야냉면   None  2013.08.16.
8892    None  우장산역 왕세숫대야냉면   None  2013.08.16.
8893    None  우장산역 왕세숫대야냉면   None  2013.08.08.

[61 rows x 4 columns]
            ItemID img_url
8833  우장산역 왕세숫대야냉면    None
8834  우장산역 왕세숫대야냉면        
8835  우장산역 왕세숫대야냉면    None
8836  우장산역 왕세숫대야냉면    None
8837  우장산역 왕세숫대야냉면    None
...            ...     ...
8889  우장산역 왕세숫대야냉면    None
8890  우장산역 왕세숫대야냉면        
8891  우장산역 왕세숫대야냉면    None
8892  우장산역 왕세숫대야냉면    None
8893  우장산역 왕세숫대야냉면    None

[61 rows x 2 columns]
            ItemID                                             rev

In [22]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_rating(4245).csv')

In [21]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_User_ID(4245).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_img(4245).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_review(4245).csv')

In [89]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


1013

In [100]:
print (data.loc[data['검색어'] == "서대문역 아웃백스테이크하우스"])

             상호명 상권업종중분류명 상권업종소분류명             도로명주소       역명          거리  \
2527  아웃백스테이크하우스       양식  패밀리레스토랑  서울특별시 중구 통일로 114  5호선 서대문  122.856647   

      카카오url    역              검색어  
2527     NaN  서대문  서대문역 아웃백스테이크하우스  


IndexError: list index out of range